# Notebook to insert recipes from csv into db

In [1]:
import psycopg2
import os
import pandas as pd

## 1. Jamie Oliver recipes

In [ ]:
recipes_2 = pd.read_csv("../raw_data/recipe_list_2010_JamieOliver.csv")

In [ ]:
len(recipes_2)

In [ ]:
recipes_2.head(5)

TODO: Ingredients are currently a string, for our database we need them to be a list
+ Strip the brackets at the end and beginning
+ replace quotation marks 
+ split the string on commas 
--> we have a list 

In [ ]:
#This is what we insert into our db for ingredients 
recipes_2.iloc[1]["ingredients"][1:-1].replace("'", "").split(",")

In [ ]:
recipes_2.isnull().sum()

In [ ]:
#Fill NaN of cuisine and dietary restrictions
recipes_2.fillna("Not specified", inplace=True)

In [ ]:
#add column for difficulty
for index, row in recipes_2.iterrows():
    if row["total_time"] < 15:
        recipes_2.loc[index, "difficulty"] = "easy"
    elif row["total_time"] < 45:
        recipes_2.loc[index, "difficulty"] = "medium"
    else:
        recipes_2.loc[index, "difficulty"] = "hard" 

In [ ]:
recipes_2.info()

## 2. BBC Recipes

In [2]:
recipes_1 = pd.read_csv("../raw_data/recipe_list_1015_bbcfood_4231.csv")

In [3]:
recipes_1.head()

,title,total_time,yields,ingredients,instructions,image,calories,url,cuisine,dietary_restrictions,num_of_ingredients
0,Apricot tarte tatin,30,6 serving(s),"['75g/2¾oz caster sugar', '40g/1½oz unsalted b...","For caramelising the apricots, you need a smal...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...,{},https://www.bbc.co.uk/food/recipes/apricot_tar...,NaN,NaN,5
1,Celebration chocolate mousse cake,0,10 serving(s),"['25g/1oz cocoa powder, plus extra for dusting...",Preheat the oven to 180C/160C Fan/Gas 4. Greas...,https://ichef.bbci.co.uk/food/ic/food_16x9_832...,{},https://www.bbc.co.uk/food/recipes/celebration...,NaN,NaN,13
2,Healthy jerk chicken wraps,60,4 serving(s),"['250ml/9fl oz white wine vinegar', '4 tbsp su...","Heat the white wine vinegar, sugar and coriand...",https://food.files.bbci.co.uk/kandl-food/3109/...,{},https://www.bbc.co.uk/food/recipes/healthy_fas...,NaN,NaN,11
3,Venison cobbler,150,6 serving(s),"['2 tbsp sunflower oil', '2 large onions, slic...",Preheat the oven to 180C/350F/Gas 4.\nHeat the...,https://food.files.bbci.co.uk/kandl-food/3109/...,{},https://www.bbc.co.uk/food/recipes/venison_cob...,NaN,NaN,18
4,Outstanding fried egg sandwich,40,1 serving(s),"['½ ripe avocado, stone removed', '½ red chill...","Scoop the avocado into a bowl and, using a for...",https://ichef.bbci.co.uk/food/ic/food_16x9_832...,{},https://www.bbc.co.uk/food/recipes/outstanding...,NaN,NaN,11


In [4]:
len(recipes_1)

4231

In [7]:
recipes_1.isnull().sum()

title                   0
total_time              0
yields                  0
ingredients             0
instructions            0
image                   0
calories                0
url                     0
cuisine                 0
dietary_restrictions    0
num_of_ingredients      0
dtype: int64

In [6]:
#Fill NaN of cuisine and dietary restrictions
recipes_1.fillna("Not specified", inplace=True)

In [8]:
#add column for difficulty
for index, row in recipes_1.iterrows():
    if row["total_time"] < 15:
        recipes_1.loc[index, "difficulty"] = "easy"
    elif row["total_time"] < 45:
        recipes_1.loc[index, "difficulty"] = "medium"
    else:
        recipes_1.loc[index, "difficulty"] = "hard" 

In [9]:
#ToDO replace calories with not  specified or so 
recipes_1.drop(columns="calories", inplace=True)

In [10]:
recipes_1["calories"] = "No information available"

In [11]:
recipes_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4231 entries, 0 to 4230
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 4231 non-null   object
 1   total_time            4231 non-null   int64 
 2   yields                4231 non-null   object
 3   ingredients           4231 non-null   object
 4   instructions          4231 non-null   object
 5   image                 4231 non-null   object
 6   url                   4231 non-null   object
 7   cuisine               4231 non-null   object
 8   dietary_restrictions  4231 non-null   object
 9   num_of_ingredients    4231 non-null   int64 
 10  difficulty            4231 non-null   object
 11  calories              4231 non-null   object
dtypes: int64(2), object(10)
memory usage: 396.8+ KB


## 3. Create a new table in our database with the new format (only once)

In [14]:
DB_PASSWORD = os.environ['DB_PASSWORD']

In [15]:
conn = psycopg2.connect(database="d1hsr1c7nk56dl", user = "iadkkqrgljveni", host = "ec2-3-230-61-252.compute-1.amazonaws.com", port = "5432", password=DB_PASSWORD)

In [16]:
cur = conn.cursor()

In [ ]:
#Creating the table only ONCE
cur.execute(
    '''CREATE TABLE COOKIT_RECIPES
    (ID INT PRIMARY KEY     NOT NULL,
    TITLE           TEXT    NOT NULL,
    DIFFICULTY      TEXT,
    PREPTIME        INT,
    NUMBER_OF_INGREDIENTS INT,
    INGREDIENTS     TEXT[]     NOT NULL,
    CUISINE         TEXT     NOT NULL,
    CALORIES        TEXT,
    LINK            TEXT       NOT NULL,
    PICTURE_URL     TEXT       NOT NULL,
    INSTRUCTIONS    TEXT);''')

In [ ]:
conn.commit()

## 4. Insert recipes into newly created table 

In [17]:
#Based on loaded dataframe
def insert_into_db(recipes):
    
    for index, row in recipes.iterrows():
        primary_key = index+4123+750
        title = row["title"]
        difficulty = row["difficulty"]
        prep_time = row["total_time"]
        number_of_ingredients = row["num_of_ingredients"]
        ingredients = recipes.iloc[index]["ingredients"][1:-1].replace("'", "").split(",")
        cuisine = row["cuisine"]
        calories = row["calories"]
        url = row["url"]
        picture_url = row["image"]
        instructions = row["instructions"]

        
        query = """INSERT INTO COOKIT_RECIPES (ID, TITLE, DIFFICULTY, PREPTIME, NUMBER_OF_INGREDIENTS, 
        INGREDIENTS, CUISINE, CALORIES, LINK, PICTURE_URL, INSTRUCTIONS)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        
        cur.execute(query, (primary_key, title, difficulty, prep_time, number_of_ingredients, ingredients, cuisine, calories, url, picture_url, instructions));
        conn.commit()

In [18]:
insert_into_db(recipes_1)